In [1]:
import pandas as pd
import numpy as np
import itertools
import sklearn.metrics as met
import scipy as sp

import sys
sys.path.insert(0, '../../notebooks/libs/')
import FeatureExtraction as FE


from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score

import sklearn.metrics as met
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler


from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn.grid_search import GridSearchCV

/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
training = pd.read_csv('../../csv/chat_based_features_training.csv')
test = pd.read_csv('../../csv/chat_based_features_test.csv')

features = ['number of conversation', 'percent of conversations started by the author', 'number of messages sent',
            'average percent of lines in conversation', 'number of characters sent by the author']

column_names = training.columns.values.tolist()[1:-1]

training_sparse_chat_based = sp.sparse.csr_matrix(training[column_names].values, dtype=float)
test_sparse_chat_based = sp.sparse.csr_matrix(test[column_names].values, dtype=float)

In [3]:
training_xml = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
test_xml = '../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

documents_training = FE.prepare_for_tf_idf(training_xml, False)
documents_test = FE.prepare_for_tf_idf(test_xml, False)

In [4]:
tfidf = TfidfVectorizer(stop_words='english', min_df=3, max_features=2500)

training_sparse_tfidf = tfidf.fit_transform(documents_training)
testing_sparse_tfidf = tfidf.transform(documents_test)

In [5]:
training_all = sp.sparse.hstack((training_sparse_tfidf, training_sparse_chat_based))

test_all = sp.sparse.hstack((testing_sparse_tfidf, test_sparse_chat_based))

In [6]:
scaler=StandardScaler(with_mean=False)
training_all=scaler.fit_transform(training_all)
test_all=scaler.transform(test_all)

In [7]:
model = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=27, reg_alpha= 1e-07)

model.fit(training_all, training[['is sexual predator']])

/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.3, max_delta_step=0, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=177, nthread=8,
       objective='binary:logistic', reg_alpha=1e-07, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

In [8]:
prediction_xgb = model.predict(test_all)

In [9]:
print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction_xgb) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction_xgb)
print 'Recall:', met.recall_score(test[['is sexual predator']], prediction_xgb)
print 'F1:', met.f1_score(test[['is sexual predator']], prediction_xgb)
print 'F0.5:', met.fbeta_score(test[['is sexual predator']], prediction_xgb, 0.5)

Accuracy:  0.999414728718
Precision:  0.875
Recall: 0.57874015748
F1: 0.696682464455
F0.5: 0.79373650108


In [10]:
xgb_all_1 = xgb.XGBClassifier(max_depth=13, n_estimators=280, learning_rate=0.028, scale_pos_weight=5, gamma=4,
                          min_child_weight=5, subsample=1, max_delta_step=3)
xgb_all_2 = xgb.XGBClassifier(max_depth=25, n_estimators=300)
xgb_all_3 = xgb.XGBClassifier(max_depth=5, n_estimators=220, learning_rate=0.007, scale_pos_weight=20, gamma=2)
xgb_all_4 = xgb.XGBClassifier(max_depth=3, n_estimators=400)
xgb_all_5 = xgb.XGBClassifier(max_depth=7, n_estimators=100)

xgb_all_1.fit(chat_based_scaled_and_tfidf_training, training[['is sexual predator']])
xgb_all_2.fit(chat_based_scaled_and_tfidf_training, training[['is sexual predator']])
xgb_all_3.fit(chat_based_scaled_and_tfidf_training, training[['is sexual predator']])
xgb_all_4.fit(chat_based_scaled_and_tfidf_training, training[['is sexual predator']])
xgb_all_5.fit(chat_based_scaled_and_tfidf_training, training[['is sexual predator']])

prediction0 = xgb_all_1.predict(chat_based_scaled_and_tfidf_test)
prediction1 = xgb_all_2.predict(chat_based_scaled_and_tfidf_test)
prediction2 = xgb_all_3.predict(chat_based_scaled_and_tfidf_test)
prediction3 = xgb_all_4.predict(chat_based_scaled_and_tfidf_test)
prediction4 = xgb_all_5.predict(chat_based_scaled_and_tfidf_test)

NameError: name 'chat_based_scaled_and_tfidf_training' is not defined

In [11]:
def bagging(vote_number, prediction_list):
    total_prediction = []
    for i in range(len(prediction_list[0])):
        voters = 0
        
        for prediction in prediction_list:
            if prediction[i] == 1:
                voters += 1
                
        if voters >= vote_number:
            total_prediction.append(1)
        else:
            total_prediction.append(0)
                
    return np.array(total_prediction)

In [ ]:
xgb_all_prediction_bagged = bagging(4, [prediction0, prediction1, prediction2, prediction3, prediction4])

print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], xgb_all_prediction_bagged) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], xgb_all_prediction_bagged)
print 'Recall:', met.recall_score(test[['is sexual predator']], xgb_all_prediction_bagged)
print 'F1:', met.fbeta_score(test[['is sexual predator']], xgb_all_prediction_bagged, 1)
print 'F0.5:', met.fbeta_score(test[['is sexual predator']], xgb_all_prediction_bagged, 0.5)

In [12]:
model1 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=27, reg_alpha= 1e-07)

model2 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=1, reg_alpha= 1e-07)

model3 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=53, reg_alpha= 1e-07)

model4 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=97, reg_alpha= 1e-07)

model5 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=171, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=1127, reg_alpha= 1e-07)

model6 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=22227, reg_alpha= 1e-07)

model7 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=24392, reg_alpha= 1e-07)

model8 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=353, reg_alpha= 1e-07)

model9 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=41255, reg_alpha= 1e-07)

model10 = xgb.XGBClassifier( learning_rate = 0.3, n_estimators=177, max_depth=4,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=8, scale_pos_weight=1,seed=5323, reg_alpha= 1e-07)

model1.fit(training_all, training[['is sexual predator']])
model2.fit(training_all, training[['is sexual predator']])
model3.fit(training_all, training[['is sexual predator']])
model4.fit(training_all, training[['is sexual predator']])
model5.fit(training_all, training[['is sexual predator']])
model6.fit(training_all, training[['is sexual predator']])
model7.fit(training_all, training[['is sexual predator']])
model8.fit(training_all, training[['is sexual predator']])
model9.fit(training_all, training[['is sexual predator']])
model10.fit(training_all, training[['is sexual predator']])

/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y 

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.3, max_delta_step=0, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=177, nthread=8,
       objective='binary:logistic', reg_alpha=1e-07, reg_lambda=1,
       scale_pos_weight=1, seed=5323, silent=True, subsample=0.8)

In [13]:
prediction1 = model1.predict(test_all)
prediction2 = model2.predict(test_all)
prediction3 = model3.predict(test_all)
prediction4 = model4.predict(test_all)
prediction5 = model5.predict(test_all)
prediction6 = model6.predict(test_all)
prediction7 = model7.predict(test_all)
prediction8 = model8.predict(test_all)
prediction9 = model9.predict(test_all)
prediction10 = model10.predict(test_all)

In [55]:
prediction_10_xgb = bagging(2, [prediction1, prediction2, prediction3, prediction4, prediction5,
                        prediction6, prediction7, prediction8, prediction9, prediction10])

print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction_10_xgb) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction_10_xgb)
print 'Recall:', met.recall_score(test[['is sexual predator']], prediction_10_xgb)
print 'F1:', met.fbeta_score(test[['is sexual predator']], prediction_10_xgb, 1)
print 'F0.5:', met.fbeta_score(test[['is sexual predator']], prediction_10_xgb, 0.5)

Accuracy:  0.999465025468
Precision:  0.854922279793
Recall: 0.649606299213
F1: 0.738255033557
F0.5: 0.804093567251


In [31]:
rf_all = RandomForestClassifier(n_estimators=230, n_jobs=8, bootstrap=False)
rf_all = rf_all.fit(training_all, np.ravel(training[['is sexual predator']]))

xgb_all = xgb.XGBClassifier(max_depth=14, n_estimators=300, learning_rate=0.2, scale_pos_weight=5, gamma=4,
                          min_child_weight=5, subsample=1)
xgb_all.fit(training_all, np.ravel(training[['is sexual predator']]))

xgb_chat_based = xgb.XGBClassifier(max_depth=25, n_estimators=300, learning_rate=0.007, scale_pos_weight=5,
                                   gamma=7, objective='binary:logistic')
xgb_chat_based.fit(training_all, np.ravel(training[['is sexual predator']]))

svm_tfidf = svm.SVC(C=0.35,kernel='linear',max_iter=100)
svm_tfidf.fit(training_sparse_tfidf, np.ravel(training[['is sexual predator']]))

svm_chat_based = svm.SVC(C=0.7454,kernel='linear',max_iter=100)
svm_chat_based.fit(training_sparse_chat_based, np.ravel(training[['is sexual predator']]))

SVC(C=0.7454, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=100, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
rf_all_predicition = rf_all.predict(test_all)
xgb_all_prediction = xgb_all.predict(test_all)
xgb_chat_based_prediction = xgb_chat_based.predict(test_all)
svm_tfidf_predicition  = svm_tfidf.predict(testing_sparse_tfidf)
svm_cb_predicition  = svm_chat_based.predict(test_sparse_chat_based)

In [ ]:
def get_file_prediction(path):
    f = open(path, 'r')
    return f.read().split(' ')

petra_svm_pred = get_file_prediction('predicted_svm.txt')
print len(petra_svm_pred)

In [56]:
ensamble_prediction = bagging(4, [xgb_all_prediction, xgb_chat_based_prediction, prediction_xgb,
                                  rf_all_predicition, svm_tfidf_predicition, svm_cb_predicition,
                                 prediction_10_xgb])

In [57]:
print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], ensamble_prediction) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], ensamble_prediction)
print 'Recall:', met.recall_score(test[['is sexual predator']], ensamble_prediction)
print 'F1:', met.fbeta_score(test[['is sexual predator']], ensamble_prediction, 1)
print 'F0.5:', met.fbeta_score(test[['is sexual predator']], ensamble_prediction, 0.5)

Accuracy:  0.999465025468
Precision:  0.910179640719
Recall: 0.59842519685
F1: 0.722090261283
F0.5: 0.824295010846


In [74]:
def write_predators_to_file(prediction, authors):
    output_handle = open('classified_preadtors_822.txt', 'w')
    bool_list = [True if p == 1 else False for p in prediction]
    predators = authors[bool_list]
    for predator in predators:
        output_handle.write(predator + '\n')
        
write_predators_to_file(ensamble_prediction, test['autor'])

In [ ]:
#data = {'p1':svm_tfidf_predicition, 'p2':rf_chat_based_predicition, 'p3':xgb_all_prediction_bagged,
#        'p4':xgb_all_prediction, 'p5':xgb_chat_based_prediction}
#s = pd.DataFrame(data)

#s.head()

In [ ]:
clf1 = xgb.XGBClassifier(max_depth=14, n_estimators=300, learning_rate=0.2, scale_pos_weight=5, gamma=4,
                          min_child_weight=5, subsample=1)
clf2 = svm.SVC(C=0.35,kernel='linear',max_iter=100)
clf3 = RandomForestClassifier(n_estimators=230, n_jobs=8, bootstrap=False)

eclf = VotingClassifier(estimators=[('xgb', clf1), ('svm', clf2), ('rf', clf3)],
                        weights=[1, 1, 1])


features = ['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']

clf1 = clf1.fit(chat_based_scaled_and_tfidf_training, np.ravel(training[['is sexual predator']]))
clf2 = clf2.fit(training[features], np.ravel(training[['is sexual predator']]))
clf3 = clf3.fit(training[features], np.ravel(training[['is sexual predator']]))

eclf = eclf.fit(training[features], np.ravel(training[['is sexual predator']]))

eclf_prediction = eclf.predict(test[features])


print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], eclf_prediction) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], eclf_prediction)
print 'Recall:', met.recall_score(test[['is sexual predator']], eclf_prediction)
print 'F1:', met.fbeta_score(test[['is sexual predator']], eclf_prediction, 1)
print 'F0.5:', met.fbeta_score(test[['is sexual predator']], eclf_prediction, 0.5)

In [ ]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = DecisionTreeClassifier(max_depth=4)
clf5 = KNeighborsClassifier(n_neighbors=7)
clf6 = SVC(kernel='rbf', probability=True)

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gn', clf3),
        ('dt', clf4), ('knn', clf5), ('svc', clf6)], voting='soft', weights=[1,1,1,2,1,2])

features = ['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']
clf1 = clf1.fit(training[features], np.ravel(training[['is sexual predator']]))
clf2 = clf2.fit(training[features], np.ravel(training[['is sexual predator']]))
clf3 = clf3.fit(training[features], np.ravel(training[['is sexual predator']]))
clf4 = clf4.fit(training[features], np.ravel(training[['is sexual predator']]))
clf5 = clf5.fit(training[features], np.ravel(training[['is sexual predator']]))
clf6 = clf6.fit(training[features], np.ravel(training[['is sexual predator']]))
eclf = eclf.fit(training[features], np.ravel(training[['is sexual predator']]))
eclf_prediction = eclf.predict(test[features])


print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], eclf_prediction) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], eclf_prediction)
print 'Recall:', met.recall_score(test[['is sexual predator']], eclf_prediction)
print 'F1:', met.fbeta_score(test[['is sexual predator']], eclf_prediction, 1)

In [ ]:
params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200],}
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(iris.data, iris.target)